## Klue/bert-base fine-tuning

huggingface에서 Klue/bert-base 모델을 받아 fine-tuning을 해보겠습니다. 


In [1]:
import random
from tqdm.notebook import tqdm, tnrange
import os

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler

from sklearn.metrics import accuracy_score

if torch.cuda.is_available():
    print("사용가능한 GPU수 : ",torch.cuda.device_count())
    device = torch.device("cuda")
else:
    print("CPU 사용")
    device = torch.device("cpu")

사용가능한 GPU수 :  1


Reproduction을 위한 Seed 고정  
출처 : https://dacon.io/codeshare/2363?dtype=vote&s_id=0

In [2]:
RANDOM_SEED = 42

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(RANDOM_SEED)

In [3]:
model_checkpoint = "klue/bert-base"
save_checkpoint_path = "./checkpoints"
batch_size = 32

huggingface 에서 tokenizer를 불러옵니다.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

huggingface 에서 model를 불러옵니다.

In [5]:
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [6]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

dataset을 가져옵니다. 

In [7]:
dataset = pd.read_csv("data/train_data.csv")
test = pd.read_csv("data/test_data.csv")

In [8]:
dataset.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


train 데이터, validation 데이터 나눕니다.

In [9]:
dataset_train, dataset_val = train_test_split(dataset,test_size = 0.2,random_state = RANDOM_SEED)

tokenize함수를 정의해줍니다.

In [10]:
def bert_tokenize(dataset,sent_key,label_key,tokenizer):
    if label_key is None :
        labels = [np.int64(0) for i in dataset[sent_key]]
    else :
        labels = [np.int64(i) for i in dataset[label_key]]
    
    sentences = tokenizer(dataset[sent_key].tolist(),truncation=True,padding=True)

    input_ids = sentences.input_ids
    token_type_ids = sentences.token_type_ids
    attention_mask = sentences.attention_mask
    
    return list([input_ids, token_type_ids, attention_mask, labels])

In [11]:
train_inputs = bert_tokenize(dataset_train,"title","topic_idx",tokenizer)
validation_inputs = bert_tokenize(dataset_val,"title","topic_idx",tokenizer)
test_inputs = bert_tokenize(test,"title",None,tokenizer)

토큰화가 잘 되었는지 확인합니다.

In [12]:
print(dataset_train["title"][0])
print(train_inputs[0][0])

인천→핀란드 항공기 결항…휴가철 여행객 분통
[2, 5108, 10948, 7288, 3662, 2470, 3646, 2048, 4117, 4542, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
for i in range(len(train_inputs)):
    train_inputs[i] = torch.tensor(train_inputs[i])
    
for i in range(len(validation_inputs)):
    validation_inputs[i] = torch.tensor(validation_inputs[i])
    
for i in range(len(test_inputs)):
    test_inputs[i] = torch.tensor(test_inputs[i])

학습에 사용될 dataloader를 만듭니다.

In [14]:
train_data = TensorDataset(*train_inputs)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(*validation_inputs)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

test_data = TensorDataset(*test_inputs)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

hyperparameter를 정의합니다. 

In [15]:
lr = 2e-5
adam_epsilon = 1e-8

epochs = 3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) 

학습을 진행합니다. 

In [16]:
train_loss_set = []
learning_rate = []

model.zero_grad()

for _ in tnrange(1,epochs+1,desc='Epoch'):
    print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
    batch_loss = 0

    # train
    for step, batch in enumerate(tqdm(train_dataloader)):
        model.train()

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_type_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()
        optimizer.zero_grad()

        batch_loss += loss.item()

    avg_train_loss = batch_loss / len(train_dataloader)


    for param_group in optimizer.param_groups:
        print("\n\tCurrent Learning rate: ",param_group['lr'])
        learning_rate.append(param_group['lr'])

    train_loss_set.append(avg_train_loss)
    print(F'\n\tAverage Training loss: {avg_train_loss}')

    # validation
    model.eval()
    eval_accuracy,nb_eval_steps = 0, 0, 0

    for batch in tqdm(validation_dataloader):
    
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_type_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)
            
        logits = logits[0].to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

        tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>


  0%|          | 0/1142 [00:00<?, ?it/s]


	Current Learning rate:  1.3333333333333333e-05

	Average Training loss: 0.38847610105200947


  0%|          | 0/286 [00:00<?, ?it/s]

...


	Validation Accuracy: 0.895134694850604
<====================== Epoch 3 ======================>


  0%|          | 0/1142 [00:00<?, ?it/s]


	Current Learning rate:  0.0

	Average Training loss: 0.15152730786562904


  0%|          | 0/286 [00:00<?, ?it/s]


	Validation Accuracy: 0.8923831849968213


학습이 끝난 뒤 예측을 진행합니다.

In [17]:
# predict
pred = []
model.eval()

for batch in tqdm(test_dataloader):
    
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_token_type_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)
    logits = logits[0].to('cpu').numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()

    for p in pred_flat:
        pred.append(p)

print(len(pred))

  0%|          | 0/286 [00:00<?, ?it/s]

9131


In [18]:
pred[:10]

[2, 3, 2, 0, 3, 0, 5, 3, 4, 4]

In [19]:
submission = pd.read_csv('data/sample_submission.csv')
submission['topic_idx'] = pred
submission.to_csv("results/klue-bert-base-fine-tuning.csv",index=False)